# Prétraitement des données de tweets

**Auteur :** Mehdi MUNIM

**Date :** 2023-11-26

**Description :**

Ce notebook prétraite les données de tweets en utilisant les fonctions définies dans le fichier `preprocessing.py`.

In [1]:
#@title 1. Importation des librairies

import pandas as pd
import sys

# Ajouter le chemin du dossier src pour importer les modules
sys.path.append('../src')
import preprocessing

## 2. Chargement des données

In [2]:
#@title 2.1 Chargement des données

# Charger les données du fichier CSV
df = pd.read_csv('../data/raw/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)

# Renommer les colonnes
df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']

# Afficher les premières lignes du DataFrame
print("Premières lignes du DataFrame :")
display(df.head())

Premières lignes du DataFrame :


,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


## 3. Prétraitement des données

In [3]:
#@title 3.1 Nettoyage du texte

# Appliquer la fonction de nettoyage du module preprocessing
df['text'] = df['text'].apply(preprocessing.clean_tweet)

# Afficher les premières lignes du DataFrame avec les tweets nettoyés
print("Premières lignes du DataFrame avec les tweets nettoyés :")
display(df.head())

Premières lignes du DataFrame avec les tweets nettoyés :


,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,awww thats bummer shoulda got david carr third...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset cant update facebook texting might cry r...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,dived many time ball managed save rest go bound
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole body feel itchy like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,behaving im mad cant see


In [ ]:
#@title 3.2 Tokenization

from transformers import BertTokenizer

# Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokeniser les tweets
X = tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors="tf")
y = df['sentiment'].map({0: 0, 4: 1})  # Convertir les labels en 0 et 1

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
#@title 3.3 Vectorisation

from sklearn.model_selection import train_test_split

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split( 1 
    df['text'], df['sentiment'], test_size=0.2, random_state=42
)

# Vectorisation TF-IDF
X_train_tfidf, X_test_tfidf = preprocessing.vectorize_tfidf(X_train, X_test)

# Vectorisation Word2Vec
X_train_w2v, X_test_w2v = preprocessing.vectorize_word2vec(X_train, X_test)

# Vectorisation Doc2Vec
X_train_d2v, X_test_d2v = preprocessing.vectorize_doc2vec(X_train, X_test)

## 4. Sauvegarde des données prétraitées

In [ ]:
#@title 4.1 Sauvegarde des données

import os

# Chemin d'accès au dossier data
data_path = '../data/processed/'

# Créer le dossier s'il n'existe pas
os.makedirs(data_path, exist_ok=True)

# Sauvegarde des données pour chaque méthode
def save_data(X_train, X_test, methode):
    with open(data_path + f'X_train_{methode}.pickle', 'wb') as f:
        pickle.dump(X_train, f)
    with open(data_path + f'X_test_{methode}.pickle', 'wb') as f:
        pickle.dump(X_test, f)

save_data(X_train_tfidf, X_test_tfidf, "tfidf")
save_data(X_train_w2v, X_test_w2v, "word2vec")
save_data(X_train_d2v, X_test_d2v, "doc2vec")

# Sauvegarde de y_train et y_test
with open(data_path + 'y_train.pickle', 'wb') as f:
    pickle.dump(y_train, f)

with open(data_path + 'y_test.pickle', 'wb') as f:
    pickle.dump(y_test, f)